## Categories
* 0: Normal behavior
* 1: Minor misbehavior
* 2: Moderate misbehavior
* 3: Serious misbehavior
* 4: Critical misbehavior

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import csv

##### All Paths

In [3]:
labels_path = r'smart-watch\smart-watch\data\training_data\labels.csv'
image_dir_path = r'smart-watch\smart-watch\data\training_data\img_dir'
model_save_path = r"smart-watch\smart-watch\data\models\misbehavior_classifier.pth"

labels_path = os.path.abspath(labels_path)
image_dir = os.path.abspath(image_dir_path)
model_save_path = os.path.abspath(model_save_path)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\haridas\AppData\Local\Temp\ipykernel_18708\2577129088.py:1: SyntaxWarning: invalid escape sequence '\s'
  labels_path = 'smart-watch\smart-watch\data\training_data\labels.csv'
C:\Users\haridas\AppData\Local\Temp\ipykernel_18708\2577129088.py:3: SyntaxWarning: invalid escape sequence '\s'
  image_dir_path = 'smart-watch\smart-watch\data\training_data\img_dir'


In [4]:

def load_labels(csv_path):
    labels = {}
    with open(csv_path, mode='r') as f:
        reader = csv.reader(f)
        for row in reader:
            filename, label = row
            labels[filename] = int(label)  # Convert the label to an integer
    return labels


In [ ]:

# Define the Dataset Class
class MisbehaviorDataset(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        """
        Args:
            image_dir (str): Path to the directory with images.
            labels (dict): A dictionary mapping image filenames to labels (e.g., {"img1.png": 0, "img2.png": 4}).
            transform: Transformations to apply to the images.
        """
        self.image_dir = image_dir
        self.labels = labels
        self.transform = transform
        self.image_filenames = list(labels.keys())

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        label = self.labels[img_name]
        img_path = os.path.join(self.image_dir, img_name)
        
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, label


# Preprocessing and DataLoader
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),         # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize (ImageNet values)
                         std=[0.229, 0.224, 0.225])
])


# Dataset and DataLoader
labels = load_labels(labels_path)
dataset = MisbehaviorDataset(image_dir=image_dir, labels=labels, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


# Load and Modify ResNet-101 for Fine-Tuning
# Load the pre-trained ResNet-101 model
model = models.resnet101(pretrained=True)

# Step 1: Freeze the convolutional layers
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers by default

# Step 2: Unfreeze the final fully connected layer (fc)
model.fc = nn.Linear(model.fc.in_features, 5)  # 5 categories: Normal, Minor, Moderate, Serious, Critical
for param in model.fc.parameters():
    param.requires_grad = True  # Only unfreeze the last layer (fc layer)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# Define Training Components
# Loss function and optimizer (only optimizing the final fully connected layer)
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)  # Only optimize the final layer

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")



# Save the Model

# Save the model to the desired location
torch.save(model.state_dict(), model_save_path)




# Inference

In [ ]:
# Load the trained model
def load_model(model_path):
    # Define the model architecture (same as used for training)
    model = models.resnet101(pretrained=False)  # ResNet-101, no pre-trained weights
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 5)  # Replace the final layer with 5 categories

    # Load the model weights
    model.load_state_dict(torch.load(model_path))

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    return model
model = load_model(model_save_path)
model.eval()

# Preprocess a single image for prediction
def preprocess_single_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)

# Predict the category of a new image
image_path = "" # Testing image-path
image_tensor = preprocess_single_image(image_path).to(device)

with torch.no_grad():
    outputs = model(image_tensor)
    _, predicted = torch.max(outputs, 1)
    print(f"Predicted category: {predicted.item()}")  # The predicted class label
